## Exporting Results

Let's first save the results from our homepage's quickstart into a plot object. 

In [1]:
from distortions.geometry import Geometry, bind_metric, local_distortions, neighborhoods
from distortions.visualization import dplot
import anndata as ad
import numpy as np
import scanpy as sc

adata = ad.AnnData(np.random.poisson(2, size=(100, 5)))
sc.pp.neighbors(adata, n_neighbors=15)
sc.tl.umap(adata)

geom = Geometry(affinity_kwds={"radius": 2}, adjacency_kwds={"n_neighbors": 15})
_, Hvv, Hs = local_distortions(adata.obsm["X_umap"], adata.X, geom)
embedding = bind_metric(adata.obsm["X_umap"], Hvv, Hs)

N = neighborhoods(adata, outlier_factor=1)
plot = dplot(embedding)\
    .mapping(x="embedding_0", y="embedding_1")\
    .inter_edge_link(N=N)\
    .geom_ellipse()

If you print plot, you will see the figure. Hovering the mouse near a point highlights its neighbors. If you double click the figure, it freezes the interactivity.

In [2]:
plot

dplot(dataset=[{'embedding_0': -0.9118266105651855, 'embedding_1': 1.0791281461715698, 'x0': -0.21846944459422…

![](https://raw.githubusercontent.com/krisrs1128/distortions-data/main/figures/quickstart.gif)

To save a specific view, call the `save()` method on the plot object. By default, it will save a file `plot.svg` to the current notebook directory. You can specify your own path, but it must be an SVG file.

In [3]:
plot.save("output.svg")

One known limitation is that you have to at least hover over the SVG before the plot will properly save. This is because anywidget loads some utilities lazily, and we can't save until some interactions have been made. The first time you run `plot.save()` (before any interactions), it might save an empty svg file to the directory.

In [4]:
embedding = adata.obsm["X_umap"].copy()
radius = 3 * np.mean(adata.obsp["distances"].data)
geom = Geometry("brute", laplacian_method="geometric", laplacian_kwds={"scaling_epps": 5}, adjacency_kwds={"n_neighbors": 15}, affinity_kwds={"radius": radius})
H, _, _ = local_distortions(embedding, adata.X, geom)
metrics = {k: H[k] for k in range(len(H))}
embedding = bind_metric(embedding, Hvv, Hs)

plot = dplot(embedding)\
    .mapping(x="embedding_0", y="embedding_1")\
    .inter_isometry(metrics=metrics)\
    .geom_ellipse()
plot

dplot(dataset=[{'embedding_0': -0.9118266105651855, 'embedding_1': 1.0791281461715698, 'x0': -0.21846944459422…

In some cases, we don't need to save the visualization but do want to extract the coordinates after local correction. In this case, we can run the `.correct()` method on the plot object. If you interact with the scatterplot, freeze the view (escape or double click), and then rerun `.correct()`, you will see a new DataFrame giving the coordinates of the updated ellipses in the `embedding_0` and `embedding_1` columns. The new ellipse axes lengths and orientations are given by the remaining columns `s0`, ..., `y0`.  The `kernel` columns provide the relatively distance between the mouse position and each sample.

In [7]:
plot.correct()

,embedding_0,embedding_1,s0,s1,x0,y0,new_angle,kernel_transform,kernel_metric,_id
0,-0.918332,1.083936,0.197455,0.137518,-0.217632,0.975561,12.575856,0.000851,4.457394e-16,0
1,-0.911819,-3.908542,3.188313,0.165688,0.199422,0.979829,168.495881,0.000058,6.607757e-22,1
2,2.030262,0.256392,0.739788,1.957376,-0.740312,-0.672126,132.236192,0.648926,1.150733e-01,2
3,0.108151,1.200447,0.339889,0.166594,-0.996363,-0.082226,94.717710,0.005019,3.183321e-12,3
4,1.865381,1.711911,0.272267,0.220325,0.040091,0.984539,177.668153,0.021147,4.228648e-09,4
...,...,...,...,...,...,...,...,...,...,...
95,-0.508276,-0.215417,0.536864,0.238637,0.162054,0.985726,170.664069,0.003400,4.542880e-13,95
96,-1.414942,-2.089723,0.805274,0.239668,0.020284,0.999684,178.837618,0.000231,6.583190e-19,96
97,1.456391,-2.687694,0.319260,0.065878,-0.999745,0.021300,88.779446,0.006576,1.229313e-11,97
98,-0.412325,-2.297009,0.589482,0.134503,-0.995513,0.094572,84.573294,0.001077,1.447029e-15,98
